## Scenario 3: Multiple Data Scientists Working on Multiple ML Models

### MLflow Setup

- **Tracking server**: Yes, remote server (GCP VM).
- **Backend store**: PostgreSQL database.
- **Artifacts store**: Google Cloud Storage (GCS) bucket.

The experiments can be explored by accessing the remote server.

The example uses GCP to host a remote server. In order to run the example, you'll need a GCP account. Follow the steps below to create a new GCP account, set up the necessary infrastructure, and launch the tracking server.

### Steps to Set Up MLflow on GCP

1. **Create a GCP Account**:
   - Go to the [Google Cloud Console](https://console.cloud.google.com/).
   - Follow the instructions to create a new GCP account and set up billing.

2. **Create a GCP VM Instance**:
   - Navigate to the Compute Engine section in the Google Cloud Console.
   - Create a new VM instance:
     - Choose a name for your instance.
     - Select a region and zone.
     - Choose a machine type (e.g., n1-standard-1).
     - Configure boot disk (e.g., Ubuntu 20.04 LTS).
   - Allow HTTP/HTTPS traffic in the "Firewall" section.
   - Create and start the instance.

3. **Set Up PostgreSQL on GCP**:
   - Optionally, you can use Cloud SQL to set up a managed PostgreSQL instance.
   - Alternatively, install PostgreSQL on your VM:
     ```sh
     sudo apt update
     sudo apt install -y postgresql postgresql-contrib
     sudo -i -u postgres
     createuser --interactive
     createdb mlflow
     ```

4. **Create a GCS Bucket**:
   - Go to the Storage section in the Google Cloud Console.
   - Create a new bucket for storing MLflow artifacts.

5. **SSH into Your VM**:
   ```sh
   gcloud compute ssh <your-vm-instance-name>

6. **Install MLflow and PostgreSQL on Your VM:**
   - sudo apt update
   - sudo apt install -y python3-pip
   - pip3 install mlflow psycopg2-binary

7. **Start MLflow server:**
   - mlflow server \
    --backend-store-uri postgresql://<username>:<password>@localhost/mlflow \
    --default-artifact-root gs://<your-gcs-bucket> \
    --host 0.0.0.0 --port 5000


In [ ]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

TRACKING_SERVER_HOST = "" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [ ]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

In [ ]:
mlflow.search_experiments()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")
    
mlflow.search_experiments()

## Interacting with the model registry


In [ ]:
from mlflow.tracking import MlflowClient


client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [ ]:
client.list_registered_models()

In [ ]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)